<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Helium_Mobius_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp

# =============================
# Símbolos
# =============================
u, v, w, zeta = sp.symbols('u v w zeta', positive=True)

# Coordenadas perimétricas → r1, r2, r12
r1 = (u + v) / 2
r2 = (u + w) / 2
r12 = (v + w) / 2

# Jacobiano
J = sp.Rational(1, 8) * u*v*w * (u+v) * (u+w) * (v+w)

# =============================
# Base Hylleraas (não simetrizada)
# =============================
def hylleraas_basis(N=2):
    B = []
    for i in range(N+1):
        for j in range(N+1 - i):
            for k in range(N+1 - i - j):
                B.append((i, j, k))
    return B

# =============================
# Função base φ_{i,j,k}
# =============================
def phi(i, j, k):
    return (r1**i) * (r2**j) * (r12**k) * sp.exp(-zeta*(r1 + r2))

# =============================
# Combinações simetrizadas
#   Singlet:  Φ⁺_{ijk} = φ_{ijk}            (i=j)
#                       (φ_{ijk}+φ_{jik})/√2 (i≠j)
#   Triplet:  Φ⁻_{ijk} = 0                   (i=j)
#                       (φ_{ijk}-φ_{jik})/√2 (i>j)
# =============================
def basis_symmetrized(N=2, sector="singlet"):
    B = []
    if sector == "singlet":
        # canónica: manter i>=j para evitar duplicados
        for (i, j, k) in hylleraas_basis(N):
            if i < j:
                continue
            B.append((i, j, k))
    else:  # triplet
        for (i, j, k) in hylleraas_basis(N):
            if i > j:
                B.append((i, j, k))
    return B

def Phi_sym(i, j, k, sector="singlet"):
    if sector == "singlet":
        if i == j:
            return phi(i, j, k)
        else:
            return (phi(i, j, k) + phi(j, i, k)) / sp.sqrt(2)
    else:  # triplet
        if i == j:
            return sp.Integer(0)
        # aqui assumimos i>j por construção da base
        return (phi(i, j, k) - phi(j, i, k)) / sp.sqrt(2)

# =============================
# Integração analítica dos momentos
#   expr = P(u,v,w) * exp(-2ζu - ζv - ζw)
# =============================
def integral_moment(expr):
    expo = sp.exp(-2*zeta*u - zeta*v - zeta*w)
    poly_part = sp.simplify(expr / expo)
    poly_expanded = sp.expand(poly_part)

    res = sp.Integer(0)
    for term in poly_expanded.as_ordered_terms():
        coeff = term.as_coeff_Mul()[0]
        mon = term/coeff
        a = sp.degree(mon, u)
        b = sp.degree(mon, v)
        c = sp.degree(mon, w)
        res += coeff * sp.factorial(a)/(2*zeta)**(a+1) \
                     * sp.factorial(b)/(zeta)**(b+1) \
                     * sp.factorial(c)/(zeta)**(c+1)
    return sp.simplify(res)

# =============================
# Matriz S para um setor dado
# =============================
def build_S_sector(N=2, sector="singlet"):
    B = basis_symmetrized(N, sector)
    K = len(B)
    S = sp.zeros(K, K)
    for a, (i, j, k) in enumerate(B):
        Phi_a = Phi_sym(i, j, k, sector)
        for b, (p, q, r) in enumerate(B):
            Phi_b = Phi_sym(p, q, r, sector)
            expr = Phi_a * Phi_b * J
            S[a, b] = integral_moment(expr)
    return B, sp.simplify(S)

# =============================
# Execução (N=2)
# =============================
N = 2
basis_sing, S_sing = build_S_sector(N=N, sector="singlet")
basis_trip, S_trip = build_S_sector(N=N, sector="triplet")

print(">>> Base singlet (i>=j), N=2:")
print(basis_sing)
print("\n>>> Matriz S_singlet (analítica):")
sp.pprint(S_sing)

print("\n>>> Base triplet (i>j), N=2:")
print(basis_trip)
print("\n>>> Matriz S_triplet (analítica):")
sp.pprint(S_trip)


>>> Base singlet (i>=j), N=2:
[(0, 0, 0), (0, 0, 1), (0, 0, 2), (1, 0, 0), (1, 0, 1), (1, 1, 0), (2, 0, 0)]

>>> Matriz S_singlet (analítica):
⎡ 25      159      591      225      795      513      153   ⎤
⎢─────   ──────   ──────   ──────   ──────   ──────   ────── ⎥
⎢    9       10       11       10       11       11       11 ⎥
⎢16⋅ζ    32⋅ζ     32⋅ζ     64⋅ζ     64⋅ζ     64⋅ζ     16⋅ζ   ⎥
⎢                                                            ⎥
⎢ 159     591      5019     795      6501     3957     1197  ⎥
⎢──────  ──────   ──────   ──────   ───────  ───────  ────── ⎥
⎢    10      11       12       11        12       12      12 ⎥
⎢32⋅ζ    32⋅ζ     64⋅ζ     64⋅ζ     128⋅ζ    128⋅ζ    32⋅ζ   ⎥
⎢                                                            ⎥
⎢ 591     5019     5985     6501    15057     17541    21465 ⎥
⎢──────  ──────   ──────   ───────  ──────   ───────  ───────⎥
⎢    11      12       13        12      13        13       13⎥
⎢32⋅ζ    64⋅ζ     16⋅ζ     128⋅ζ    64